In [ ]:
import os

# The current working directory needs to be in explainable_TGCNN
print(os.getcwd())
os.chdir('..\\')
print(os.getcwd())

import pandas as pd
import numpy as np
import tensorflow as tf
from src import create_fake_patients, whole_model_demographics_gradcam, graph_plot, ga, gc, utils
import statistics

In [ ]:
second_TGCNN_layer = True
demo = True

include_drugs = True
max_timesteps=100

stride = 1
filter_size = 4

run_name='hip_1999_to_one_year_advance_model'
years_in_advance = "5"

if include_drugs:
    max_event_codes = 518
else:
    max_event_codes = 512
hip_or_knee = 'hip'
num_filters=16

# fake mapping dataframe for the ReadCodes and the corresponding descriptions
read_code_map_df = pd.read_csv('fake_read_code_descriptions.csv')

model = whole_model_demographics_gradcam.TGCNN_Model(num_filters=16, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model.load_weights('hip_1999_to_one_year_advance_model1_CNN_layer')


num_pats = 5
cv_patients = create_fake_patients.create_fake_patient_df(num_pats, 99, max_event_codes)

In [ ]:
replacement_true_lst, max_w_filt_lst, filt_nums = ga.get_act_metric_per_feat(model, num_filters, num_pats, 
                                                                             cv_patients, max_event_codes, hip_or_knee,
                                                                             'max', max_timesteps)

mean_activation_df = ga.act_diff(replacement_true_lst, max_w_filt_lst, filt_nums)

In [ ]:
sparsity_lst = []
for i in range(num_pats):
    if (i % 500) == 0 and (i !=0):
        print(f"{(((i+1)/num_pats)*100):.2f}% Complete")
        print(f"Sparsity mean +- std: {statistics.mean(sparsity_lst)}$\pm${statistics.stdev(sparsity_lst)}")
    input_3d, input_4d, demo_tensor, outcome, outcome_bin = utils.return_pat_from_df(cv_patients, max_event_codes, hip_or_knee, i, max_timesteps)
    
    dense_tensor = tf.sparse.to_dense(input_3d)
    dense_tensor= tf.transpose(dense_tensor, perm=[2, 1, 0])
    dense_tensor = np.flip(dense_tensor, axis=0)
    logits = model(input_4d, demo_tensor, training=False)
    
    metric = 'largest' # 'mean', 'largest', 'median', 'single'
    # Get the entire patient's history in a DataFrame
    edges_df = graph_plot.create_edges_df_gc(dense_tensor)
    
    # Get the node positions for the graph
    pos_df = graph_plot.create_position_df_gc(edges_df)
    pos_list = graph_plot.generate_pos_sequence(pos_df['max_codes_per_visit'].max())
    pos_df = graph_plot.map_y_coord_to_node(pos_df, pos_list)
    
    weighted_f_map = ga.choose_feat_map(model, metric, mean_activation_df)
    
    # map the timesteps to their relevant features
    timestep_ave_w_df = gc.calc_timestep_weights(1, filter_size, weighted_f_map, max_timesteps)
    
    
    # map the timestep values to colour the edges of the graph
    read_code_pos_df = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_w_df)
    
    read_code_pos_df['bin_infl'] = read_code_pos_df['perc_timestep_infl'].where(read_code_pos_df['perc_timestep_infl'] == 0, 1)
    sparsity = 1 - read_code_pos_df['bin_infl'].sum()/len(read_code_pos_df)
    sparsity_lst.append(sparsity)
    

print("Mean", statistics.mean(sparsity_lst))
print("std", statistics.stdev(sparsity_lst))
print(f"Sparsity mean +- std: {statistics.mean(sparsity_lst)}$\pm${statistics.stdev(sparsity_lst)}")